In [1]:
# User Input
#    ↓
# Enricher LLM → parses natural language query into smart intent & goal metadata
#    ↓
# Orchestrator LLM → decides what tools to call (MarketDataTool, NewsTool, etc.)
#    ↓
# Parallel Tool Execution (runs only the selected ones)
#    ↓
# Synthesizer LLM → builds final human-readable report
#    ↓
# ✅ Done


In [158]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="Gemma2-9b-It")

In [149]:
from typing_extensions import TypedDict
from pydantic import BaseModel , Field

class Refined(BaseModel):
    
    company : str = Field(default = "NotMentioned", description = "particular company mentioned if any " )
    intent  : str = Field(description = "intent of the query")
    goal    : str = Field(description = "goal of the query")
    detail_level : str = Field(default="Basic" , description = "detail level what user demands.")


class ToolToCall(BaseModel):
    name : str = Field( description = "Name of the tool to call" )
    reason : str = Field( description = "contains reason why should this tool be called" )

class SocialContent(BaseModel):
    name : str = Field( description = "Title of the content" )
    reason : str = Field( description = "social media posts about a company" )

class MessageState(TypedDict):
    
    user_query : str
    refined_query : Refined
    tools_to_call : list[ToolToCall]
    social_details : list[SocialContent]
    market_data : str
    news_data : str

In [150]:
from langchain_core.messages import HumanMessage , SystemMessage

def refining_query(state : MessageState):
    prompt = [
        SystemMessage( content = "You are a smart assistant that extracts user intent from stock market related queries." ),
        HumanMessage( content = f"""
        User asked : { state["user_query"] }
        Return structured content in form of :
          company : particular company mentioned if any 
          intent  : intent of the query
          goal : goal of the query
          detail_level : detail level what user demands.

        Note : if no company metion let company name be "NotMentioned"
        """ )
    ]
    structured_llm = llm.with_structured_output(Refined)
    response = structured_llm.invoke(prompt)
    return {"refining_query" : response}
    

In [190]:
input_query = {"user_query" : "Tell me about the Tesla stock market today "}
refined_query = refining_query(input_query)
print(refined_query)

{'refining_query': Refined(company='Tesla', intent='query', goal='information', detail_level='Basic')}


In [191]:
tools_available = [

    { "name" : "MarketDataTool" , "description": "Fetch stock price and financial data of last 30 days."  },
    { "name" : "NewsTool", "description" : " Get latest news headlines about a company / related " },
    { "name" : "CompetitorTool", "description": "Give me detailed analysis and situation of competitors of a company and tell relative position of current company in market" },
    { "name" : "SocialMediaTool", "description": "Fetch social media posts about a company from reddit.com or twiiter or stocktwiss.com" }
    { "name" : "Comparsion between t"}

]

# TOOLS 

SocialMedia Tool

In [192]:
from langchain_community.tools.tavily_search import TavilySearchResults

def SocialMediaTool(state : MessageState):

    """
    Fetch social media posts about a company from reddit.com or twiiter or stocktwiss.com
    using tavily search
    """

    search_tool = TavilySearchResults(max_results=15)
    results = search_tool.invoke(f"""
                site:reddit.com OR site:twitter.com OR site:stocktwits.com "{state["refining_query"].company}" 
    """)
    content = []
    for r in results:
        if r.get('content') and len(r.get('content').strip()) > 100:
            content.append(
                SocialContent(
                    name=r.get('title'),
                    reason=r.get('content')
                )
            )
        
    return { "social_details" : content }


In [193]:

socialcontent = SocialMediaTool(refined_query)
print(socialcontent)

{'social_details': [SocialContent(name='Tesla 2025 Q1 Quarterly Update Mega thread : r/teslamotors - Reddit', reason="r/teslamotors  \nThe original and largest Tesla community on Reddit! An unofficial forum of owners and enthusiasts. See r/TeslaLounge for relaxed posting, and user experiences! Tesla Inc. is an energy + technology company originally from California and currently headquartered in Austin, Texas. Their mission is to accelerate the world's transition to sustainable energy. They produce vertically integrated electric vehicles, batteries, solar, and AI software and hardware solutions. [...] r/teslamotors\nThe original and largest Tesla community on Reddit! An unofficial forum of owners and enthusiasts. See r/TeslaLounge for relaxed posting, and user experiences! Tesla Inc. is an energy + technology company originally from California and currently headquartered in Austin, Texas. Their mission is to accelerate the world's transition to sustainable energy. They produce verticall

MarketDataTool

In [194]:
import yfinance as yf
from datetime import datetime, timedelta


def MarketData(state : MessageState):

    """
    Fetches stock data for the last 30 days using yfinance.
    """
    if state["refining_query"].company == "NotMentioned":
        return { "market_data" : " Not available " }
    
    ticker_response = llm.invoke( f"""
                    What is the stock ticker symbol for the company named {state["refining_query"].company}?
                    If no company exists, just return a random company's ticker symbol.
                    The output should only be the ticker symbol, nothing else.
    """
    )

    ticker = ticker_response.content.strip() 
    
    stock = yf.Ticker(ticker)
    data = stock.history(period="30d")

    summary = f"Stock data for {ticker} i.e {state['refining_query'].company} over the last 30 days:\n"
    for date,row in data.iterrows():
        date_str = date.strftime("%b %d, %Y")
        open_price = row["Open"]
        close_price = row["Close"]
        volume = row["Volume"]    
        summary += f"Date: {date_str}, Open: ${open_price:.2f}, Close: ${close_price:.2f}, Volume: {volume}\n"
    return { "market_data" : summary }

In [207]:
MarketData(refined_query)

{'market_data': 'Stock data for TSLA i.e Tesla over the last 30 days:\nDate: Mar 14, 2025, Open: $247.31, Close: $249.98, Volume: 100242300.0\nDate: Mar 17, 2025, Open: $245.06, Close: $238.01, Volume: 111900600.0\nDate: Mar 18, 2025, Open: $228.16, Close: $225.31, Volume: 111477600.0\nDate: Mar 19, 2025, Open: $231.61, Close: $235.86, Volume: 111993800.0\nDate: Mar 20, 2025, Open: $233.35, Close: $236.26, Volume: 99028300.0\nDate: Mar 21, 2025, Open: $234.99, Close: $248.71, Volume: 132728700.0\nDate: Mar 24, 2025, Open: $258.08, Close: $278.39, Volume: 169079900.0\nDate: Mar 25, 2025, Open: $283.60, Close: $288.14, Volume: 150361500.0\nDate: Mar 26, 2025, Open: $282.66, Close: $272.06, Volume: 153629800.0\nDate: Mar 27, 2025, Open: $272.48, Close: $273.13, Volume: 162572100.0\nDate: Mar 28, 2025, Open: $275.58, Close: $263.55, Volume: 123809400.0\nDate: Mar 31, 2025, Open: $249.31, Close: $259.16, Volume: 134008900.0\nDate: Apr 01, 2025, Open: $263.80, Close: $268.46, Volume: 1464869

NewsTool

In [212]:
import requests

os.environ["GNEWS_API_KEY"] = os.getenv("GNEWS_API_KEY")

def NewsTool(state : MessageState):
        """
        Tool that helps to fetch news related to a company 
        """
        url = f'https://gnews.io/api/v4/search?q={state["refining_query"].company}&token={os.environ["GNEWS_API_KEY"]}'
        response = requests.get(url)
        data = response.json()
        # print("API Response:", data) 
        if data.get("articles"):
            headlines = [article["title"] for article in data["articles"]]
            return "\n".join(headlines)
        else:
            print("hi")
            return "No news found."


In [213]:
NewsTool( refined_query )

API Response: {'totalArticles': 93497, 'articles': [{'title': 'Big techs perdem US$ 2,66 trilhões desde a posse de Trump', 'description': 'Só o valor de mercado da Tesla, de Elon Musk, caiu 32,9% de 20 de janeiro até esta 6ª feira (25.abr). Leia no Poder360.', 'content': 'Só o valor de mercado da Tesla, de Elon Musk, caiu 32,9% de 20 de janeiro até 6ª feira (25.abr)\nO “Magnificent Seven”, grupo das 7 gigantes do setor de tecnologia, perdeu US$ 2,66 trilhões em valor de mercado desde a posse do presidente dos Estados U... [5318 chars]', 'url': 'https://www.poder360.com.br/poder-economia/big-techs-perdem-us-266-trilhoes-desde-a-posse-de-trump/', 'image': 'https://static.poder360.com.br/2025/04/Donald-Trump-15-848x477.png', 'publishedAt': '2025-04-26T09:00:20Z', 'source': {'name': 'Poder360', 'url': 'https://www.poder360.com.br'}}, {'title': 'le NGP de Xpeng nous a bluffés', 'description': "Vous pensiez que Tesla était seul en conduite autonome ? C'est faux évidemment. On a testé le NGP 

"Big techs perdem US$ 2,66 trilhões desde a posse de Trump\nle NGP de Xpeng nous a bluffés\nKasupke ärgert sich über Berlins Arbeitssenatorin\nL'agence américaine assouplit certaines règles de sécurité pour les véhicules autonomes, l'action Tesla s'envole\nThe left blindly hates Elon Musk, but Americans owe him thanks\nDrôle d'ironie dans les résultats trimestriels de Tesla\nLES ACTIONS EN VUE A WALL STREET CE VENDREDI : Intel, Alphabet, Tesla, Gilead, T-Mobile US, SLB\nTesla-Aktie an der NASDAQ von Musk-Versprechen angetrieben: Tesla-Zahlen enttäuschen auf ganzer Linie\nBYD dobra lucro e supera Tesla no 1º trimestre de 2025\nΟ Μασκ αφήνει την Ουάσιγκτον και πιάνει την Tesla: Τι θα απογίνει το DOGE και το όραμα των περικοπών"